In [2]:
# data
import nfl_data_py as nfl

# data loading and plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# models
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier, plot_importance

# interpretation
import shap
from interpret import show

# pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, nan_euclidean_distances

pd.set_option('display.max_columns', None)  # None zeigt alle Spalten
pd.set_option('display.max_rows', None)  # Alle Zeilen anzeigen, vorsichtig bei großen DataFrames
pd.set_option('display.width', 1000)  # Breite anpassen

In [14]:
df = nfl.import_ids()

In [16]:
df[df['merge_name']=='jayden daniels']

,twitter_username,espn_id,fleaflicker_id,merge_name,draft_pick,height,rotowire_id,cbs_id,draft_round,cfbref_id,position,fantasypros_id,sportradar_id,mfl_id,swish_id,ktc_id,stats_global_id,college,draft_year,draft_ovr,yahoo_id,db_season,pfr_id,fantasy_data_id,age,name,weight,stats_id,rotoworld_id,pff_id,gsis_id,nfl_id,team,birthdate,sleeper_id
2,NaN,4426348.0,NaN,jayden daniels,2.0,76.0,17692.0,3123337.0,1.0,jayden-daniels-1,QB,22902.0,bab8c65a-9fff-410d-8e34-dc2f636d890e,16581,1161228.0,1552.0,0.0,LSU,2024.0,2.0,40896.0,2024,DaniJa02,NaN,24.0,Jayden Daniels,210.0,40896.0,NaN,NaN,00-0039910,NaN,WAS,2000-12-18,11566.0


In [25]:
df = df[df['club_code'] == 'CIN']
df = df[df['position'] == 'QB']

KeyError: 'club_code'

In [ ]:
df.head(40)

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
3510,2023,CIN,1.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3511,2023,CIN,1.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3573,2023,CIN,2.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3574,2023,CIN,2.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3659,2023,CIN,3.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3680,2023,CIN,3.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3726,2023,CIN,4.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3727,2023,CIN,4.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3786,2023,CIN,5.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3788,2023,CIN,5.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
